In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. Siêu tham số
batch_size = 64
epochs = 5
lr = 0.01
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_ds = datasets.MNIST('.', train=True, download=True, transform=transform)
test_ds = datasets.MNIST('.', train=False, download=True, transform=transform)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=1000, shuffle=False)

# 3. Định nghĩa mô hình CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

model = CNN().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# 4. Hàm huấn luyện
def train():
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# 5. Hàm đánh giá
def test():
    model.eval()
    test_loss = correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    print(f'Test loss: {test_loss/len(test_loader):.4f}, Accuracy: {100*correct/len(test_ds):.2f}%')

# 6. Vòng huấn luyện và đánh giá
for epoch in range(1, epochs+1):
    train()
    print(f'Epoch {epoch}:')
    test()

# 7. Lưu mô hình
torch.save(model.state_dict(), 'mnist_cnn.pt')


/home/thanhhuy/miniconda3/envs/torchdl/lib/python3.13/site-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 1:
Test loss: 0.2714, Accuracy: 92.53%
Epoch 2:
Test loss: 0.1984, Accuracy: 94.08%
Epoch 3:
Test loss: 0.1514, Accuracy: 95.77%
Epoch 4:
Test loss: 0.1274, Accuracy: 96.21%
Epoch 5:
Test loss: 0.1083, Accuracy: 96.63%
